# Calculate yield anomalies and plot them
- Script to take the processed crop yield data and calculate crop yield anomalies. The script calculates three different types of anomalies to enable sensitivity analyses, and writes out new objects accordingly
- In a second step these yield anomalies are written out to gridded netCDF objects for ease of use in other applications as this is the standard format for other gridded yield products


In [1]:
from scipy import signal
from scipy import ndimage
import netCDF4
import os, json
import warnings
import regionmask
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import pandas as pd
import geopandas as gpd
from tools import save_hdf
from tools import CreateLinkAdmin
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None

In [2]:
#define a moving average function
def moving_average(a, smooth) :
    n=smooth*2+1
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [3]:
#read in the crop yield data
df = pd.read_hdf('./data/crop/adm_crop_production_ALL.hdf')
#read in the FEWS admins
shape = gpd.read_file('./data/shapefile/adm_current.shp').to_crs("EPSG:4326")
#merge the two
df = pd.read_hdf('./data/crop/adm_crop_production_ALL.hdf')
df = df.merge(shape[['FNID','ADMIN0','ADMIN1','ADMIN2']], left_on='fnid', right_on='FNID')
df = df.rename(columns={'ADMIN1':'admin1','ADMIN2':'admin2','season_name':'season'})
df['admin'] = df['fnid'].apply(lambda x: x[2:8])
cols = ['fnid','country','admin','product','season','harvest_month','harvest_year','indicator','value']
df = df[cols]

### Calculate anomalies and grid
Below is the main portion of the script, where anomalies are calculated and written out as both a table and as a gridded netCDF product

In [4]:
#use FNIDs and years to create an empty dataframe
fnids = df.fnid.unique()
yrs = np.sort(df.harvest_year.unique())
dfEmpty = pd.DataFrame(columns = yrs, index=fnids,dtype=float)

#get a table of FNIDs and geometries
fnGeos = shape[['FNID','geometry']].drop_duplicates()
fnGeos.set_index('FNID',inplace=True)

#make an empty grid
lon = np.arange(-20,55,0.1)
lat = np.arange(40,-40,-0.1)

cps = {
    'maize1':{'Somalia':['Maize','Deyr'],
            'Malawi':['Maize','Main'],
            'Kenya':['Maize','Short'],
            'Burkina Faso':['Maize','Main'],
            'Mali':['Maize','Main'],
            'Chad':['Maize','Main'],
            'South Africa':['Maize (Yellow)','Summer'],
            'Niger':['Maize','Main season'],
            'Zambia':['Maize','Annual'],
             'Angola':['Maize','Main'],
              'Mozambique':['Maize','Main harvest']
             },
    
    'maize2':{'Somalia':['Maize','Gu'],
            'Kenya':['Maize','Long']},
    
    'wheat':{'Kenya':['Wheat','Annual'],
            'Mali':['Wheat','Main'],
            'Chad':['Wheat','Main'],
            'South Africa':['Wheat','Winter']}
}

dfPrd = dfEmpty.copy()
dfArea = dfEmpty.copy()
dfYld = dfEmpty.copy()
dfYldGauAbs = dfEmpty.copy()
dfYldSm5 = dfEmpty.copy()
dfYldGau = dfEmpty.copy()
dfPrdGau = dfEmpty.copy()
dfAreaGau = dfEmpty.copy()


for icr in ['maize1','maize2','wheat']:
    ncPath = '/Users/wanders7/Documents/Code/Project/NASA_GSCD/gscd/data/gridded/'+icr+'.nc'
    ncf = netCDF4.Dataset(ncPath,'w',format='NETCDF4')
    #create dimensions                            
    ncf.createDimension('lon',lon.size)
    ncf.createDimension('lat',lat.size)
    ncf.createDimension('yr',yrs.size)
    #create variables
    ncf.createVariable('latitude','f4',('lat'))
    ncf.createVariable('longitude','f4',('lon'))
    ncf.createVariable('year','i8',('yr'))
    #fill variables lat/lon
    ncf.variables['latitude'][:]=lat
    ncf.variables['longitude'][:]=lon    
    ncf.variables['year'][:]=yrs
    #create the yield variables and objects
    ncf.createVariable('yieldAnom_pct','f4',('yr','lat','lon'))
    ncf.createVariable('prodAnom_pct','f4',('yr','lat','lon'))
    ncf.createVariable('areaAnom_pct','f4',('yr','lat','lon'))
    ncf.createVariable('yieldAnom_abs','f4',('yr','lat','lon'))
    ncf.createVariable('yield','f4',('yr','lat','lon'))
    ncf.createVariable('production','f4',('yr','lat','lon'))
    ncf.createVariable('area','f4',('yr','lat','lon'))
    
    
    area = np.ones([yrs.size,lat.size,lon.size])*np.nan
    prd = np.ones([yrs.size,lat.size,lon.size])*np.nan
    yld = np.ones([yrs.size,lat.size,lon.size])*np.nan
    yldAnom = np.ones([yrs.size,lat.size,lon.size])*np.nan
    yldAnomAbs = np.ones([yrs.size,lat.size,lon.size])*np.nan
    prdAnom = np.ones([yrs.size,lat.size,lon.size])*np.nan
    areaAnom = np.ones([yrs.size,lat.size,lon.size])*np.nan
    
    for icx in cps[icr].keys():
        ipx = cps[icr][icx][0]
        isx = cps[icr][icx][1]
        ifnds = df.loc[(df['product']==ipx)&(df.country==icx)&(df.season==isx)].fnid.unique()
        
        for ifnx in ifnds:
            shp = fnGeos.loc[ifnx].geometry
            
            ylds = df.loc[(df['product']==ipx)&(df.country==icx)&(df.season==isx)&
                   (df.fnid==ifnx)&(df.indicator=='yield')].sort_values(by='harvest_year')
            
            prds = df.loc[(df['product']==ipx)&(df.country==icx)&(df.season==isx)&
                   (df.fnid==ifnx)&(df.indicator=='production')].sort_values(by='harvest_year')

            areas = df.loc[(df['product']==ipx)&(df.country==icx)&(df.season==isx)&
                   (df.fnid==ifnx)&(df.indicator=='area')].sort_values(by='harvest_year')

            #check for duplicate values, which indicates multiple production systems. 
            # If present, sum the area and production and recalculate yield
            if ((np.size(ylds['harvest_year'][ylds['harvest_year'].duplicated()])>0)|
                (np.size(prds['harvest_year'][prds['harvest_year'].duplicated()])>0)|
                (np.size(areas['harvest_year'][areas['harvest_year'].duplicated()])>0)):
                cols = ['fnid','country','admin','product','season',
                        'harvest_month','harvest_year','indicator','value']
                #group area and production
                areas = areas[cols].groupby(cols[:-1]).sum()
                prds = prds[cols].groupby(cols[:-1]).sum()
                #reindex
                indexCols = cols[:-2]
                prds = prds.reset_index()
                prds.set_index(indexCols,inplace=True)
                areas = areas.reset_index()
                areas.set_index(indexCols,inplace=True)
                #write to the yield df
                ylds = prds['value']/areas['value']
                ylds=ylds.reset_index()
                prds=prds.reset_index()
                areas=areas.reset_index()
                ylds['indicator']='yield'
                
            if ylds.value.size<5:
                print('fewer than 5 years for '+ifnx+', '+ipx+', '+isx)
                continue
            
            dfPrd.loc[ifnx][prds.harvest_year] = prds.value
            dfArea.loc[ifnx][areas.harvest_year] = areas.value
            dfYld.loc[ifnx][ylds.harvest_year] = ylds.value
            
            iYld = dfYld.loc[ifnx][ylds.harvest_year].values.astype(float)
            iyldAbs = dfYld.loc[ifnx][ylds.harvest_year].values.astype(float)
            iprdAbs = dfPrd.loc[ifnx][prds.harvest_year].values.astype(float)
            iareaAbs = dfArea.loc[ifnx][areas.harvest_year].values.astype(float)

            #To account for a missing values in locations we can drop and re-weight the remaining values by counting non-zero values        
            yldCount = np.copy(iYld)
            yldCount[~np.isnan(yldCount)]=1
            yldCount[np.isnan(yldCount)]=0
            iYld[np.isnan(iYld)]=0

            prdCount = np.copy(iprdAbs)
            prdCount[~np.isnan(prdCount)]=1
            prdCount[np.isnan(prdCount)]=0
            iprdAbs[np.isnan(iprdAbs)]=0
            
            areaCount = np.copy(iareaAbs)
            areaCount[~np.isnan(areaCount)]=1
            areaCount[np.isnan(areaCount)]=0
            iareaAbs[np.isnan(iareaAbs)]=0
            
            #divide by the count to re-weight based on the # of values going into each smoothing filter
            yldSm5Exp = moving_average(np.array(iYld),2)/moving_average(np.array(yldCount),2)
            
            yldGauExp = ndimage.gaussian_filter1d(iYld,3)/ndimage.gaussian_filter1d(yldCount,3)
            prdGauExp = ndimage.gaussian_filter1d(iprdAbs,3)/ndimage.gaussian_filter1d(prdCount,3)
            areaGauExp = ndimage.gaussian_filter1d(iareaAbs,3)/ndimage.gaussian_filter1d(areaCount,3)
            
            stYldGauAbs = (iYld-yldGauExp)
            #percent yield anom = (yld obs - yld exp)/yld exp
            stYldGau = (iYld-yldGauExp)/yldGauExp
            stPrdGau = (iprdAbs-prdGauExp)/prdGauExp
            stAreaGau = (iareaAbs-areaGauExp)/areaGauExp
            stYldSm5 = np.array(iYld[2:-2]-yldSm5Exp)/yldSm5Exp
            
            #add nans back in
            stPrdGau[np.isnan(dfPrd.loc[ifnx][prds.harvest_year].values.astype(float))]=np.nan
            stAreaGau[np.isnan(dfArea.loc[ifnx][areas.harvest_year].values.astype(float))]=np.nan
            stYldGau[np.isnan(dfYld.loc[ifnx][ylds.harvest_year].values.astype(float))]=np.nan
            stYldSm5[np.isnan(dfYld.loc[ifnx][ylds.harvest_year].values[2:-2].astype(float))]=np.nan
            stYldGauAbs[np.isnan(dfYld.loc[ifnx][ylds.harvest_year].values.astype(float))]=np.nan
            
            #write the empty data frames
            dfYldGauAbs.loc[ifnx][ylds.harvest_year.values] = stYldGauAbs
            dfYldGau.loc[ifnx][ylds.harvest_year.values] = stYldGau
            dfPrdGau.loc[ifnx][prds.harvest_year.values] = stPrdGau
            dfAreaGau.loc[ifnx][areas.harvest_year.values] = stAreaGau
            dfYldSm5.loc[ifnx][ylds.harvest_year.values][2:-2] = stYldSm5
            
            #find the location mask
            msk = np.array(regionmask.Regions([fnGeos.loc[ifnx].geometry]).mask(lon,lat))
            #assign the yield time series to the relevant grid points based on the mask
            prdAnom.T[msk.T==0,...] = dfPrdGau.loc[ifnx].values
            areaAnom.T[msk.T==0,...] = dfAreaGau.loc[ifnx].values
            yldAnom.T[msk.T==0,...] = dfYldGau.loc[ifnx].values
            yldAnomAbs.T[msk.T==0,...] = dfYldGauAbs.loc[ifnx].values
            
            yld.T[msk.T==0,...] = dfYld.loc[ifnx].values
            prd.T[msk.T==0,...] = dfPrd.loc[ifnx].values
            area.T[msk.T==0,...] = dfArea.loc[ifnx].values 
            
            
    #write the yield objects into the netCDF
    ncf['yield'][:] = yld
    ncf['production'][:] = prd
    ncf['area'][:] = area
    ncf['yieldAnom_pct'][:] = yldAnom
    ncf['prodAnom_pct'][:] = prdAnom
    ncf['areaAnom_pct'][:] = areaAnom
    ncf['yieldAnom_abs'][:] = yldAnomAbs
    
    ncf.close();del ncf
    

fewer than 5 years for SO1990A22502, Maize, Deyr


/var/folders/pk/_ym78shs7tx06w00zbcwxwgc0000gp/T/ipykernel_69046/2594005666.py:178: UserWarning: No gridpoint belongs to any region. Returning an all-NaN mask.
  msk = np.array(regionmask.Regions([fnGeos.loc[ifnx].geometry]).mask(lon,lat))


fewer than 5 years for KE2013A108, Maize, Short
fewer than 5 years for KE2013A118, Maize, Short
fewer than 5 years for KE2013A123, Maize, Short
fewer than 5 years for KE2013A124, Maize, Short
fewer than 5 years for KE2013A125, Maize, Short
fewer than 5 years for KE2013A128, Maize, Short
fewer than 5 years for KE2013A129, Maize, Short
fewer than 5 years for KE2013A130, Maize, Short
fewer than 5 years for KE2013A133, Maize, Short
fewer than 5 years for KE2013A135, Maize, Short
fewer than 5 years for KE2013A147, Maize, Short
fewer than 5 years for ML2001A107, Maize, Main
fewer than 5 years for NE2012A20516, Maize, Main season
fewer than 5 years for NE2012A20603, Maize, Main season
fewer than 5 years for NE2012A20607, Maize, Main season
fewer than 5 years for NE2012A20611, Maize, Main season
fewer than 5 years for NE2012A20614, Maize, Main season
fewer than 5 years for NE2012A20616, Maize, Main season
fewer than 5 years for NE2012A20805, Maize, Main season
fewer than 5 years for NE2012A208